In [2]:
import pyarrow as pa
import pyarrow.dataset as ds
import os
import math

BYTES_PER_TYPE = {
    "int8": 1, "int16": 2, "int32": 4, "int64": 8,
    "uint8": 1, "uint16": 2, "uint32": 4, "uint64": 8,
    "float": 4, "float16": 2, "float32": 4, "float64": 8,
    "bool": 1,
}

STRING_DEFAULT_BYTES = 16

def patch_schema(schema):
    fields = []
    for field in schema:
        if pa.types.is_decimal(field.type):
            fields.append(pa.field(field.name, pa.string()))
        else:
            fields.append(field)
    return pa.schema(fields)


def estimate_array_size(arr, col_type):
    n = len(arr)

    if pa.types.is_integer(col_type) or pa.types.is_floating(col_type):
        dtype = col_type.to_pandas_dtype()
        return n * BYTES_PER_TYPE.get(dtype, 8)

    if pa.types.is_boolean(col_type):
        return n * 1

    if pa.types.is_decimal(col_type):
        return n * 16

    if pa.types.is_string(col_type):
        # Sample 5000
        sample = arr.slice(0, min(5000, n)).to_pylist()
        total_len = sum(len(x) for x in sample if x is not None)
        non_null = sum(1 for x in sample if x is not None)
        avg_len = (total_len / non_null) if non_null > 0 else STRING_DEFAULT_BYTES
        return int(n * avg_len)

    return n * 16


INPUT_DIR = "D:\\zman\\graph\\data"
results = []

for file in os.listdir(INPUT_DIR)[10:]:

    if file.endswith(".csv"):
        continue

    path = os.path.join(INPUT_DIR, file)

    base = ds.dataset(path, format="parquet", partitioning="hive")
    patched_schema = patch_schema(base.schema)
    dataset = ds.dataset(path, format="parquet", partitioning="hive", schema=patched_schema)

    print(f"\n=== Estimasi LMDB untuk: {file} ===")

    num_rows = dataset.count_rows()
    print("Rows:", num_rows)

    total_bytes = 0

    for col in dataset.schema.names:

        col_type = dataset.schema.field(col).type
        col_bytes = 0

        # Scanner hanya untuk 1 kolom
        scanner = dataset.scanner(columns=[col], batch_size=75_000)

        for batch in scanner.to_batches():
            arr = batch.column(0)
            col_bytes += estimate_array_size(arr, col_type)

        total_bytes += col_bytes

        print(f"{col:30} | {str(col_type):25} | {col_bytes/1024/1024:10.2f} MB")

    lmdb_bytes = int(total_bytes * 1.2)
    gb = math.ceil(lmdb_bytes / (1024**3))

    print(f"\nTOTAL ESTIMATED SIZE: {total_bytes/1024/1024/1024:.2f} GB")
    print(f"RECOMMENDED LMDB MAP_SIZE: {gb} GB\n")

    results.append((file, gb))

print("\n===================== RINGKASAN MAP_SIZE =====================")
for file, gb in results:
    print(f"{file:40} : {gb} GB")
print("==============================================================")




=== Estimasi LMDB untuk: node_transaksi ===
Rows: 24502104
id_trx                         | string                    |    1093.32 MB
id                             | string                    |     155.26 MB
seq                            | string                    |     140.20 MB
auxtrc                         | string                    |      93.47 MB
dsctrc                         | string                    |     185.51 MB
src                            | string                    |     257.79 MB
amt                            | string                    |     303.33 MB
dst                            | string                    |     260.11 MB
curr                           | string                    |      93.47 MB
cd                             | string                    |      23.37 MB
channel                        | string                    |     934.68 MB

TOTAL ESTIMATED SIZE: 3.46 GB
RECOMMENDED LMDB MAP_SIZE: 5 GB


===================== RINGKASAN MAP_SIZE =========

In [1]:
import os
import pyarrow.dataset as ds
import pyarrow as pa
import lmdb
import json
from tqdm import tqdm


###########################################
# CONFIG
###########################################

NODE_TABLES = {
    # "nasabah": "D:\\zman\\graph\\data\\node_nasabah",          # Parquet directory (partitioned)
    # "pekerja": "D:\\zman\\graph\\data\\node_pekerja",
    # "pinjaman": "D:\\zman\\graph\\data\\node_pinjaman",
    # "simpanan": "D:\\zman\\graph\\data\\node_simpanan",
    "transaksi": "D:\\zman\\graph\\data\\node_transaksi",
}

ID_COLUMN = {
    # "nasabah": "cif",
    # "pekerja": "pn",
    # "pinjaman": "acctno",
    # "simpanan": "acctno",
    "transaksi": "id_trx"
}

LMDB_PATH = "lmdb_node_mapping"
# MAP_SIZE = 1024 * 1024 * 1024 * 50   # 50GB, adjust if necessary

MAP_SIZE = {
    # "nasabah": 1024 * 1024 * 1024 * 23,
    # "pekerja": 1024 * 1024 * 1024 * 1,
    # "pinjaman": 1024 * 1024 * 1024 * 3,
    # "simpanan": 1024 * 1024 * 1024 * 34,
    "transaksi": 1024 * 1024 * 1024 * 5
}

BATCH_SIZE = 75_000  # Read parquet in chunks


###########################################
# UTILS
###########################################

def encode(x: str) -> bytes:
    return x.encode()

def decode(b: bytes) -> str:
    return b.decode()


###########################################
# MAIN MAPPER FUNCTION
###########################################
def patch_schema(schema):
    fields = []
    for field in schema:
        if pa.types.is_decimal(field.type):
            # Convert DECIMAL to STRING
            fields.append(pa.field(field.name, pa.string()))
        else:
            fields.append(field)
    return pa.schema(fields)


def build_node_mapping():
    os.makedirs(LMDB_PATH, exist_ok=True)

    envs = {}
    for node_type in NODE_TABLES.keys():
        envs[node_type] = lmdb.open(
            os.path.join(LMDB_PATH, f"{node_type}.lmdb"),
            map_size=MAP_SIZE[node_type],
            subdir=True,
            lock=True,
            readonly=False,
            max_dbs=1,
        )

    counters = {nt: 0 for nt in NODE_TABLES.keys()}

    # Loop setiap tabel node
    for node_type, folder in NODE_TABLES.items():
        print(f"\n=== Processing Node: {node_type} ===")
        base = ds.dataset(folder, format="parquet", partitioning="hive")
        patched_schema = patch_schema(base.schema)
        dataset = ds.dataset(folder, format="parquet", partitioning="hive", schema=patched_schema)
        
        env = envs[node_type]
        id_col = ID_COLUMN[node_type]
        counter = counters[node_type]

        # buka transaction manual
        txn = env.begin(write=True)
        
        for batch in tqdm(dataset.to_batches(batch_size=30_000)):
            batch_dict = batch.to_pydict()   # hindari ArrowDecimal error
            
            if id_col not in batch_dict:
                raise KeyError(f"Kolom {id_col} tidak ditemukan dalam batch. Schema: {batch.schema}")
            ids = batch_dict[id_col]

        # for batch in dataset.to_batches(batch_size=BATCH_SIZE):
        #     col = batch.column(batch.schema.get_field_index(id_col))
        #     ids = col.to_pylist()

            for node_id in ids:
                if node_id is None:
                    continue

                key = encode(str(node_id))

                if txn.get(key) is None:
                    txn.put(key, encode(str(counter)))
                    counter += 1

            # Commit setiap batch utk mencegah long transaction
            txn.commit()
            txn = env.begin(write=True)

        # Final commit & close
        txn.commit()

        counters[node_type] = counter
        print(f"Total mapped: {counter:,} node {node_type}")

    print("\nAll mapping completed successfully!")
    return counters




###########################################
# RUN
###########################################

if __name__ == "__main__":
    result = build_node_mapping()
    print("\nFinal Node Counts:")
    print(json.dumps(result, indent=4))


=== Processing Node: transaksi ===


817it [2:38:31, 11.64s/it]


Total mapped: 12,516,002 node transaksi

All mapping completed successfully!

Final Node Counts:
{
    "transaksi": 12516002
}


In [ ]:
import lmdb

def inspect_lmdb(path, max_sample=5):
    env = lmdb.open(path, readonly=True, lock=False)
    total_keys = 0
    none_values = 0
    samples = []

    with env.begin() as txn:
        cursor = txn.cursor()

        for key, val in cursor:
            total_keys += 1

            # val None tidak akan muncul pada LMDB, tapi val bisa empty
            if val is None or len(val) == 0:
                none_values += 1

            if len(samples) < max_sample:
                try:
                    samples.append((key.decode(), int.from_bytes(val, "little")))
                except:
                    samples.append((key.decode(), val))

    env.close()

    print("===================================================")
    print(f"LMDB Path             : {path}")
    print("---------------------------------------------------")
    print(f"Total Unique Keys     : {total_keys:,}")
    print(f"Nilai 'None' / Empty  : {none_values:,}")
    print("---------------------------------------------------")
    print("Contoh Key-Value:")
    for s in samples:
        print("  ", s)
    print("===================================================")

    return total_keys, none_values, samples



inspect_lmdb("D:/zman/graph/notebook/lmdb_node_mapping/nasabah.lmdb")
inspect_lmdb("D:/zman/graph/notebook/lmdb_node_mapping/pekerja.lmdb")
inspect_lmdb("D:/zman/graph/notebook/lmdb_node_mapping/simpanan.lmdb")
inspect_lmdb("D:/zman/graph/notebook/lmdb_node_mapping/pinjaman.lmdb")
inspect_lmdb("D:/zman/graph/notebook/lmdb_node_mapping/transaksi.lmdb")

LMDB Path             : D:/zman/graph/notebook/lmdb_node_mapping/nasabah.lmdb
---------------------------------------------------
Total Unique Keys     : 12,270,075
Nilai 'None' / Empty  : 0
---------------------------------------------------
Contoh Key-Value:
   ('       ', 14414838833951025)
   (' 002257', 57381655819064)
   (' 002258', 15540712970926387)
   (' 002259', 14981087305544759)
   (' 002260', 14694110576587320)
LMDB Path             : D:/zman/graph/notebook/lmdb_node_mapping/pekerja.lmdb
---------------------------------------------------
Total Unique Keys     : 6,250
Nilai 'None' / Empty  : 0
---------------------------------------------------
Contoh Key-Value:
   ('103016', 892941621)
   ('103033', 859387189)
   ('104542', 875706165)
   ('112567', 892876595)
   ('112616', 808924468)
LMDB Path             : D:/zman/graph/notebook/lmdb_node_mapping/simpanan.lmdb
---------------------------------------------------
Total Unique Keys     : 15,636,712
Nilai 'None' / Empty  : 0

(12516002,
 0,
 [('100051710001423023570100066050210000000000', 13570413971387698),
  ('10005370800162352350104698850430000000000', 15262549498016308),
  ('1000537100008230276701002132500120000000000', 14128944302864437),
  ('100105271200152123050500000472401046696536', 14135558619411249),
  ('1001052736001221230200500000461701044504530', 14126753902900024)])